In [3]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from math import exp,sqrt
from scipy import optimize as op
from numpy.linalg import inv,norm
from numpy import transpose as tr
from pathlib import Path

# Data Preprocessing

In [52]:
main_data = pd.read_csv('./04_cricket_1999to2011.csv')

data = main_data
data = data[['Over','Runs','Total.Out']]
# Nx50 Matrix holding history data 
Runs_wickets = list()


inning = np.zeros(100)
i=0
for d in data.itertuples():
    if d[1]==1 or i==50:
        while i!=50:
            inning[i],inning[i+50] = inning[i-1],inning[50+i-1]
            i+=1
        Runs_wickets.append(inning)
        inning = np.zeros(100)
        i=0
    inning[i],inning[i+50] = inning[i-1]+d[2],d[3]
    i+=1

Runs_wickets = np.array(Runs_wickets)     
Runs_wickets = Runs_wickets[1:,:]


## Match till now, Overs = 30

In [124]:
#Additional Variables
k = 10
intervention_point = 30

def mRSC(X,data,intervention_point,k):

    U,S,V = np.linalg.svd(data)
    S_ = np.diag(S[:k])
    M = U[:,:k]@S_@V[:k,:]    

    M_k = np.zeros(shape=(M.shape[0],2*intervention_point))
    M_k[:,:intervention_point],M_k[:,intervention_point:] = M[:,:intervention_point],M[:,50:50+intervention_point] 

    #xopt = op.least_squares(lambda v : X-v@M_k,np.zeros(M_k.shape[0]))
    opt_result = op.least_squares(lambda v : X-v@M_k,np.zeros(M_k.shape[0]))
    xopt = opt_result.x
    print(M.shape)
    result = np.int16(np.round(xopt@M))
    print(result)  

X = np.round(np.linspace(0,150,30))
Y = np.round(np.linspace(0,5,30))
Z = np.zeros(60)
Z[:30],Z[30:] = X,Y
#print(Z)
#print(Runs_wickets.shape[0])
mRSC(Z,cdata,30,25)  

(200, 100)
[  1   5  10  16  21  26  31  36  41  47  52  57  62  67  73  78  83  88
  93  98 103 108 114 119 124 129 134 140 145 150 154 157 159 159 160 163
 166 171 177 183 188 192 199 204 208 211 210 212 219 226   0   0   1   1
   1   1   1   1   1   2   2   2   2   2   2   3   3   3   3   3   3   3
   4   4   4   4   4   5   5   5   6   6   6   6   7   7   7   7   7   8
   8   8   8   9   9  10  10  10  10  10]


In [101]:
cdata = Runs_wickets[np.random.randint(low=0,high=2845,size=200),:]

ValueError: matmul: Input operand 1 has a mismatch in its core dimension 0, with gufunc signature (n?,k),(k,m?)->(n?,m?) (size 200 is different from 100)

In [6]:
main_data = pd.read_csv('./04_cricket_1999to2011.csv')

main_data.columns

Index(['Match', 'Date', 'Innings', 'Over', 'Runs', 'Total.Runs',
       'Innings.Total.Runs', 'Runs.Remaining', 'Total.Out',
       'Innings.Total.Out', 'Outs.Remaining', 'Wickets.in.Hand', 'Run.Rate',
       'Innings.Run.Rate', 'Run.Rate.Required', 'Initial.Run.Rate.Required',
       'Target.Score', 'Day-night', 'At.Bat', 'Fielding', 'Home.Team',
       'Away.Team', 'Stadium', 'Country', 'Total.Overs', 'Winning.Team',
       'Toss.Winner', 'at.bat.wins', 'at.bat.won.toss', 'at.bat.at.home',
       'at.bat.bat.first', 'chose_bat_1st', 'chose_bat_2nd', 'forced_bat_1st',
       'forced_bat_2nd', 'new.game', 'Error.In.Data', 'common.support'],
      dtype='object')

## mRSC

9.656703768073534
5.186955646234871
2.879119831939226
1.7314195251210276
1.1340273811871353
0.40538596273842875
